**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_numerical -p 8888:8888 -p 8786:8786 -p 8787:8787 -d palmoreck/jupyterlab_numerical:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_numerical
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

---

# El módulo de `multiprocessing`

La implementación más utilizada de Python, [Cpython](https://en.wikipedia.org/wiki/CPython) no utiliza múltiples cores por default. Se recomienda leer la discusión de la liga anterior en el apartado *Design* sobre el [Global INterpreter Lock: GIL](https://en.wikipedia.org/wiki/Global_interpreter_lock) y el por qué CPython no soporta ejecución *multithreaded* o *multiprocesses* (aunque el módulo `multiprocessing` mitiga cuestiones importantes en el *shared memory programming* como la comunicación entre procesos).

El módulo `multiprocessing` nos permite realizar procesamientos basados en procesos o threads para compartir trabajo y datos. Se recomienda usar este módulo para el *shared memory programming* (ver [2.2.Sistemas_de_memoria_compartida](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.2.Sistemas_de_memoria_compartida.ipynb)) y para trabajos que son demandantes de CPU. Para paralelizar trabajos demandantes en I/O no se recomienda utilizar.

**Otro módulo en Python para procesamiento utilizando los cores de tu máquina es [concurrent.features](https://docs.python.org/3/library/concurrent.futures.html) que provee el comportamiento principal de `multiprocessing`**. Ver [liga](https://stackoverflow.com/questions/38311431/concurrent-futures-processpoolexecutor-vs-multiprocessing-pool-pool?noredirect=1&lq=1) y [liga](https://stackoverflow.com/questions/20776189/concurrent-futures-vs-multiprocessing-in-python-3) para más sobre `concurrent.futures` y `concurrent.futures` vs `multiprocessing`.

## Nota sobre el GIL y `multiprocessing`

Aunque en Python los threads son nativos del sistema operativo (esto es, no son simulados son realmente threads del sistema operativo), están limitados por el *global interpreter lock, GIL*, de modo que un sólo thread interactúe con un objeto Python en un único tiempo.

Al usar el módulo `multiprocessing` ejecutamos en paralelo un número de **interpretadores Python** (CPython), cada uno con su propio espacio de memoria privada y su propio GIL que se ejecutan en un instante. 

# Ejemplos

Por default *multiprocessing* usará tantos cores como disponibles se encuentren en tu máquina y se puede configurar a que utilice un número definido de cores.

**Referencias**

1. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.
